# Exploring the billing datasets

```Do not run in poetry shell```

NOT TESTED AFTER REMAKE OF BRONZE

In [9]:
# Imports
import os
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [10]:
# Set variables
CONNECTION_STRING = os.environ.get("STORAGE_CONNECTION_STRING")
CONTAINERNAME = os.environ.get("STORAGE_CONTAINER_NAME_1")
BLOBNAME = os.environ.get("STORAGE_BLOB_NAME")

In [11]:
def create_sas():
    CONTAINERNAME = os.environ.get("STORAGE_CONTAINER_NAME_1")
    BLOBNAME = os.environ.get("STORAGE_BLOB_NAME")
    ACCOUNTNAME = os.environ.get("STORAGE_ACCOUNT_NAME")
    QUERYSTRING = os.environ.get("BLOB_QUERY_STRING")
    
    sas_url = f"https://{ACCOUNTNAME}.blob.core.windows.net/{CONTAINERNAME}/cloudlink/azure/{BLOBNAME}{QUERYSTRING}"
    return sas_url
 
    
sas_url = create_sas()

In [12]:
df = pd.read_parquet(sas_url)
df_len, df_width = df.shape
print('Rows: ' +str(df_len),'\nColumns: ' + str(df_width))

Rows: 1755 
Columns: 65


In [13]:
# clean az data
df = df.replace(r'\s+',np.nan,regex=True).replace('',np.nan)
before = len(df.columns)
# Drop all columns where all values is NaN
df = df.dropna(axis=1,how='all')
after = len(df.columns)
removed = before-after
print("Removed "+ str(removed) + " blank columns!")

Removed 23 blank columns!


In [14]:
df.columns

Index(['invoiceId', 'billingAccountId', 'billingProfileId', 'invoiceSectionId',
       'servicePeriodEndDate', 'servicePeriodStartDate', 'date',
       'serviceFamily', 'productOrderId', 'consumedService', 'meterId',
       'meterCategory', 'meterSubCategory', 'meterRegion', 'ProductId',
       'SubscriptionId', 'subscriptionName', 'publisherType', 'publisherName',
       'resourceGroupName', 'ResourceId', 'resourceLocation', 'location',
       'effectivePrice', 'quantity', 'unitOfMeasure', 'chargeType',
       'billingCurrency', 'pricingCurrency', 'costInBillingCurrency',
       'costInPricingCurrency', 'costInUsd', 'paygCostInBillingCurrency',
       'paygCostInUsd', 'exchangeRatePricingToBilling', 'exchangeRateDate',
       'isAzureCreditEligible', 'PayGPrice', 'frequency', 'pricingModel',
       'unitPrice', 'provider'],
      dtype='object')

In [15]:
for column in df:
    x = df
    x = x[column].drop_duplicates().dropna()
    print("The column '" + column + "' has " + str(x.shape[0]) + " unique rows")

The column 'invoiceId' has 1 unique rows
The column 'billingAccountId' has 1 unique rows
The column 'billingProfileId' has 2 unique rows
The column 'invoiceSectionId' has 2 unique rows
The column 'servicePeriodEndDate' has 1 unique rows
The column 'servicePeriodStartDate' has 1 unique rows
The column 'date' has 15 unique rows
The column 'serviceFamily' has 4 unique rows
The column 'productOrderId' has 2 unique rows
The column 'consumedService' has 6 unique rows
The column 'meterId' has 25 unique rows
The column 'meterCategory' has 2 unique rows
The column 'meterSubCategory' has 1 unique rows
The column 'meterRegion' has 2 unique rows
The column 'ProductId' has 16 unique rows
The column 'SubscriptionId' has 2 unique rows
The column 'subscriptionName' has 2 unique rows
The column 'publisherType' has 1 unique rows
The column 'publisherName' has 1 unique rows
The column 'resourceGroupName' has 14 unique rows
The column 'ResourceId' has 26 unique rows
The column 'resourceLocation' has 3 uni

In [23]:
subscription_colums = ["SubscriptionId","subscriptionName", "billingAccountId", "billingAccountName", "billingProfileId", "billingProfileName"]

subscription_dim = pd.DataFrame(df,columns=subscription_colums, copy=True)
subscription_dim.index = subscription_dim["SubscriptionId"]
subscription_dim.drop_duplicates("SubscriptionId", inplace=True)
subscription_dim.columns


Index(['subscriptionName', 'SubscriptionId', 'billingAccountId',
       'billingAccountName', 'billingProfileId', 'billingProfileName'],
      dtype='object')

In [24]:
product_colums = ["ProductId","ProductName", "resourceLocation", "meterSubCategory", "consumedService"]

product_dim = pd.DataFrame(df,columns=product_colums, copy=True)
product_dim.index = product_dim["ProductId"]
product_dim.drop_duplicates("ProductId", inplace=True)
product_dim.columns

Index(['ProductId', 'ProductName', 'resourceLocation', 'meterSubCategory',
       'consumedService'],
      dtype='object')

In [26]:
usage_colums = ["quantity", "effectivePrice", "paygCostInBillingCurrency", "date", "ProductId", "SubscriptionId"]

usage_fact = pd.DataFrame(df,columns=usage_colums, copy=True)
usage_fact.index = usage_fact["ProductId"]
usage_fact = usage_fact[usage_fact['effectivePrice'] != 0]
usage_fact.columns

Index(['quantity', 'effectivePrice', 'paygCostInBillingCurrency', 'date',
       'ProductId', 'SubscriptionId'],
      dtype='object')